<a href="https://colab.research.google.com/github/newsun87/google-colab-line_bot_learning/blob/main/linebot_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### flask
https://www.jianshu.com/p/438dc54dc627

安裝套件及 ngrok 驗證

In [3]:
!pip install flask_ngrok pyngrok
!pip install Flask==2.0.3
!pip install line-bot-sdk
!ngrok authtoken 2Sh3hFOuE9JYeMNTsS6EzBxHPIL_6xdGTubFt7b9kSeUDFVYH

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=0090c3a79c57d128e5b661eded75bb4bb6ada484a3fdce8d183e5b9138d878cb
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: Flask
    Found existing installation: Flask 2.2.5
    Uninstalling Flask-2.2.5:
      Successfully uninstalled Flask-2.2.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.6/730.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.9 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


#### 發送樣板訊息
1. Confirm 樣板
2. Buttons 樣板
3. Carsousel 樣板
4. ImageCarsousel 樣板

建立 LINE bot 框架伺服器，回應 Confirem template

In [15]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *
import json
import os, time, requests

# 設定 LINE Channel access token'
access_token = 'hea6hI/LTc4FCcgIlR51IbcAjPWqy9QtsG5XX9Ce+4GwdfJH1EKl5/YOT3Hw4+aZOGWZSKcKPmEnUqzE4tOoBjPdvyVJJjIHwm9RN9/cWbwT7Je2GW7uNtaTSKysDrcW8F/BSEFJpywQo5Z9DXry6gdB04t89/1O/w1cDnyilFU='
# 設定 LINE Channel access token
channel_secret = '77ff5430fad4531107ca11f1c495e7dd'

def get_ngrokurl():
    os.system("ps aux | grep ngrok.exe | awk '{print $2}' | xargs kill -9")
    os.system("nohup ngrok http 5000&")
    time.sleep(3)
    public_url = json.loads(requests.get('http://localhost:4040/api/tunnels').text)['tunnels'][0]['public_url']
    return public_url

ngrok_url = get_ngrokurl()
print(ngrok_url)
# 你的 LINE Channel access token'
access_token = 'hea6hI/LTc4FCcgIlR51IbcAjPWqy9QtsG5XX9Ce+4GwdfJH1EKl5/YOT3Hw4+aZOGWZSKcKPmEnUqzE4tOoBjPdvyVJJjIHwm9RN9/cWbwT7Je2GW7uNtaTSKysDrcW8F/BSEFJpywQo5Z9DXry6gdB04t89/1O/w1cDnyilFU='
# 你的 LINE Channel access token
channel_secret = '77ff5430fad4531107ca11f1c495e7dd'
# Channel Access Token
line_bot_api = LineBotApi(access_token)
# Channel Secret
line_handler = WebhookHandler(channel_secret)
# 設定 webhook url
line_bot_api.set_webhook_endpoint(ngrok_url+'/callback')

# 建立 confirm 樣板板中的按鈕選項
option1 = MessageAction(
                    label = '確認',  # 顯示按鈕名稱
                    text = 'Yes', # 點擊後，用戶端發送的訊息文字
                )
option2 = MessageAction(
                    label = '不訂',  # 顯示按鈕名稱
                    text = 'No'  # 點擊後，用戶端發送的訊息文字
                )

# 建立 confirm 樣板
confirm_template = ConfirmTemplate(
     text = '你要訂晚餐嗎?', # 你要問的問題，或是文字敘述
     actions = [option1, option2]
)

# 監聽所有來自 LINE Bot Channel 的 Post Request
app = Flask(__name__)
# 監聽所有來自 LINE Bot Channel 的 Post Request
@app.route("/callback", methods=['POST'])
def linebot():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # 取得傳來的訊息文字
    body = request.get_data(as_text=True)
    # 轉換訊息內容成 JSON 物件
    json_data = json.loads(body)
    print(f"訊息內容 {json_data}")
    # 驗證訊息正確性
    try:
        line_handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 處理訊息
@line_handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
   if event.message.text == 'confirm':
    # 建立 confirm 樣板訊息
     confirm_template_message = TemplateSendMessage(
        alt_text = '我是一個確認模板', # 發送到群組的通知訊息
        template = confirm_template
    )
     line_bot_api.reply_message(event.reply_token, confirm_template_message)
   if event.message.text == 'Yes':
    message = TextSendMessage(text='馬上幫你訂!')
    line_bot_api.reply_message(event.reply_token,  message)
   if event.message.text == 'No':
    message = TextSendMessage(text='那就不訂喔!')
    line_bot_api.reply_message(event.reply_token,  message)

if __name__ == "__main__":
  app.run()

https://8f40-34-23-139-6.ngrok-free.app


<ipython-input-15-d64ea6bf8d6b>:28: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
<ipython-input-15-d64ea6bf8d6b>:30: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_handler = WebhookHandler(channel_secret)
<ipython-input-15-d64ea6bf8d6b>:32: LineBotSdkDeprecatedIn30: Call to deprecated method set_webhook_endpoint. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).set_webhook_endpoint(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_a

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470350339519021395', 'text': 'confirm'}, 'webhookEventId': '01H8WR4HM7H4DG05RMV9JPJ3G5', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693181953164, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': '18b6020317404aa4940758817ad5855b', 'mode': 'active'}]}


<ipython-input-15-d64ea6bf8d6b>:78: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, confirm_template_message)
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 00:19:14] "POST /callback HTTP/1.1" 200 -


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470350347622154354', 'text': 'No'}, 'webhookEventId': '01H8WR4P45MMMJJ53KW2M3ZRE9', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693181958034, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': 'c688a389945c474ead152ea46f3e41fd', 'mode': 'active'}]}


<ipython-input-15-d64ea6bf8d6b>:84: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token,  message)
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 00:19:19] "POST /callback HTTP/1.1" 200 -


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470350351514468353', 'text': 'Yes'}, 'webhookEventId': '01H8WR4RB8YMYXYH8A4W7GP6B3', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693181960228, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': 'da6a1abb6a644be68eb054bad52abf65', 'mode': 'active'}]}


<ipython-input-15-d64ea6bf8d6b>:81: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token,  message)
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 00:19:21] "POST /callback HTTP/1.1" 200 -


建立 LINE bot 框架伺服器，回應 Buttons 樣板訊息

In [20]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *
import json
import os, time, requests
from datetime import datetime, timedelta

def get_ngrokurl():
    os.system("ps aux | grep ngrok.exe | awk '{print $2}' | xargs kill -9")
    os.system("nohup ngrok http 5000&")
    time.sleep(3)
    public_url = json.loads(requests.get('http://localhost:4040/api/tunnels').text)['tunnels'][0]['public_url']
    return public_url

ngrok_url = get_ngrokurl()
print(ngrok_url)
# 你的 LINE Channel access token'
access_token = 'hea6hI/LTc4FCcgIlR51IbcAjPWqy9QtsG5XX9Ce+4GwdfJH1EKl5/YOT3Hw4+aZOGWZSKcKPmEnUqzE4tOoBjPdvyVJJjIHwm9RN9/cWbwT7Je2GW7uNtaTSKysDrcW8F/BSEFJpywQo5Z9DXry6gdB04t89/1O/w1cDnyilFU='
# 你的 LINE Channel access token
channel_secret = '77ff5430fad4531107ca11f1c495e7dd'
# Channel Access Token
line_bot_api = LineBotApi(access_token)
# Channel Secret
line_handler = WebhookHandler(channel_secret)
# 設定 webhook url
line_bot_api.set_webhook_endpoint(ngrok_url+'/callback')

# 建立 Buttons 樣板板中的按鈕選項
option1 = PostbackAction(
                    label = '打開電燈',  # 在按鈕選項的名稱
                    text = '開燈',
                    data = 'turn_on' # 用戶回呼文字 (看不到)
                )
option2 = MessageAction(
                    label = '現在日期時間', # 在按鈕選項的名稱
                    # 顯示現在的時間
                    text = 'time'
                )
option3 = URIAction(
                    label = 'Google',   # 在按鈕選項的名稱
                    uri = 'https://www.google.com.tw/'  # 跳轉到的網站
                )

# 建立 Buttons 樣板
buttons_template = ButtonsTemplate(
    thumbnail_image_url = 'https://i.imgur.com/GDsd8KJ.jpg',
    title = '按鈕樣板測試',  # 樣板標題名稱
    text = '功能選單',  # 選單文字敘述
    actions = [option1, option2, option3]
)

# 監聽所有來自 LINE Bot Channel 的 Post Request
app = Flask(__name__)
# 監聽所有來自 LINE Bot Channel 的 Post Request
@app.route("/callback", methods=['POST'])
def linebot():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # 取得傳來的訊息文字
    body = request.get_data(as_text=True)
    # 轉換訊息內容成 JSON 物件
    json_data = json.loads(body)
    print(f"訊息內容 {json_data}")
    # 驗證訊息正確性
    try:
        line_handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 處理訊息
@line_handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
   if event.message.text == 'buttons':
    buttons_template_message = TemplateSendMessage(
         alt_text = '我是一個按鈕選單模板',  # 發送到群組的通知訊息
         template = buttons_template
    )
    line_bot_api.reply_message(event.reply_token,  buttons_template_message)
   if event.message.text == 'time':
    # 加上 8 小時得到台灣時間
    taipei_time = datetime.now() + timedelta(hours=8)
    timenow = taipei_time.strftime("%Y-%m-%d %H:%M:%S")
    message = TextSendMessage(text = timenow)
    line_bot_api.reply_message(event.reply_token,  message)

# 處理 Postback 事件
@line_handler.add(PostbackEvent)
def handle_postback_message(event):
    postback = event.postback.data # 取得 Postback 事件的資料
# -------------------------------------
    if postback == 'turn_on':
       message = TextSendMessage(text='電燈已打開!')
       line_bot_api.reply_message(event.reply_token, message)

if __name__ == "__main__":
  app.run()

https://2193-34-83-245-156.ngrok-free.app


<ipython-input-20-f2562c4ddd77>:24: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
<ipython-input-20-f2562c4ddd77>:26: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_handler = WebhookHandler(channel_secret)
<ipython-input-20-f2562c4ddd77>:28: LineBotSdkDeprecatedIn30: Call to deprecated method set_webhook_endpoint. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).set_webhook_endpoint(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_a

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470375275444568501', 'text': 'buttons'}, 'webhookEventId': '01H8X6A42558PZAQGXNEECBA71', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693196816058, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': 'bed17ad939804a4186a4d246563f4d2b', 'mode': 'active'}]}


<ipython-input-20-f2562c4ddd77>:81: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token,  buttons_template_message)
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 04:26:57] "POST /callback HTTP/1.1" 200 -


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'postback', 'postback': {'data': 'turn_on'}, 'webhookEventId': '01H8X6A8TPRJ85HG4PHEYMCE84', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693196821294, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': '883471b2841c4fbd85f7e4b49c4c158a', 'mode': 'active'}]}


<ipython-input-20-f2562c4ddd77>:96: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, message)
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 04:27:02] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 04:27:02] "POST /callback HTTP/1.1" 200 -


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470375284052066883', 'text': '開燈'}, 'webhookEventId': '01H8X6A8TRJM1PMKQXTFMT58RV', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693196821294, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': '4765856f71ee4e898d9d6ddca14aec2e', 'mode': 'active'}]}
訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470375291668922530', 'text': 'time'}, 'webhookEventId': '01H8X6AD6GAQD3DZDSXY0REXSR', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693196825754, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': 'd0c5165cfca94178938f27939e7fe7ff', 'mode': 'active'}]}


<ipython-input-20-f2562c4ddd77>:87: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token,  message)
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 04:27:06] "POST /callback HTTP/1.1" 200 -


建立 LINE bot 框架伺服器，回應 Carousel template

In [18]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *
import json
import os, time, requests
from datetime import datetime, timedelta

def get_ngrokurl():
    os.system("ps aux | grep ngrok.exe | awk '{print $2}' | xargs kill -9")
    os.system("nohup ngrok http 5000&")
    time.sleep(3)
    public_url = json.loads(requests.get('http://localhost:4040/api/tunnels').text)['tunnels'][0]['public_url']
    return public_url

ngrok_url = get_ngrokurl()
print(ngrok_url)
# 你的 LINE Channel access token'
access_token = 'hea6hI/LTc4FCcgIlR51IbcAjPWqy9QtsG5XX9Ce+4GwdfJH1EKl5/YOT3Hw4+aZOGWZSKcKPmEnUqzE4tOoBjPdvyVJJjIHwm9RN9/cWbwT7Je2GW7uNtaTSKysDrcW8F/BSEFJpywQo5Z9DXry6gdB04t89/1O/w1cDnyilFU='
# 你的 LINE Channel access token
channel_secret = '77ff5430fad4531107ca11f1c495e7dd'
# Channel Access Token
line_bot_api = LineBotApi(access_token)
# Channel Secret
line_handler = WebhookHandler(channel_secret)
# 設定 webhook url
line_bot_api.set_webhook_endpoint(ngrok_url+'/callback')

# 建立 Carousel 樣板中的卡片
column1 = CarouselColumn(
    thumbnail_image_url = 'https://i.imgur.com/VoqMKOs.jpg',
    title = '我是狗狗',
    text = '想餵我吃東西嗎？',
    actions = [
                PostbackAction(
                     label = '想',
                     data = 'Yes'
                ),
                PostbackAction(
                     label = '不想',
                     data = 'No'
                ),
                URIAction(
                     label = '這是我的網址',
                     uri = 'https://wepet.tw/'
                )
    ]
)
column2 = CarouselColumn(
     thumbnail_image_url = 'https://i.imgur.com/Dve0c11.jpg',
     title = '我是貓貓',
     text = '想餵我吃東西嗎？',
     actions = [
                PostbackAction(
                    label = '想',
                    data = 'Yes'
                ),
                PostbackAction(
                    label = '不想',
                    data = 'No'
                ),
                URIAction(
                    label = '這還是我的網址 ',
                    uri = 'https://wepet.tw/'
                )
     ]
)
# 建立 Carousel 樣板
columns = [column1, column2] # 收集卡片
carousel_template = CarouselTemplate(columns=columns)

# 監聽所有來自 LINE Bot Channel 的 Post Request
app = Flask(__name__)
# 監聽所有來自 LINE Bot Channel 的 Post Request
@app.route("/callback", methods=['POST'])
def linebot():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # 取得傳來的訊息文字
    body = request.get_data(as_text=True)
    # 轉換訊息內容成 JSON 物件
    json_data = json.loads(body)
    print(f"訊息內容 {json_data}")
    # 驗證訊息正確性
    try:
        line_handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 處理訊息
@line_handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if event.message.text == 'carousel':
      # 建立 Carousel 樣板訊息
      carousel_template_message = TemplateSendMessage(
        alt_text='Carousel Template 訊息',
        template = carousel_template
      )
      line_bot_api.reply_message(event.reply_token,  carousel_template_message)

# 處理 Postback 事件
@line_handler.add(PostbackEvent)
def handle_postback_message(event):
    postback = event.postback.data # 取得 Postback 事件的資料
# -------------------------------------
    if postback == 'Yes':
       message = TextSendMessage(text='謝謝餵食，已經吃飽了!')
    if postback == 'No':
       message = TextSendMessage(text='沒關係，反正我也不餓!')
    line_bot_api.reply_message(event.reply_token, message)

if __name__ == "__main__":
  app.run()

https://e036-34-83-245-156.ngrok-free.app


<ipython-input-18-1fc05361ae3b>:24: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
<ipython-input-18-1fc05361ae3b>:26: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_handler = WebhookHandler(channel_secret)
<ipython-input-18-1fc05361ae3b>:28: LineBotSdkDeprecatedIn30: Call to deprecated method set_webhook_endpoint. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).set_webhook_endpoint(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_a

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470374307802382407', 'text': 'carousel'}, 'webhookEventId': '01H8X5RH2QJ5BAP4K61C4YA58M', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693196239477, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': 'fe86abcf29ba44bcb1cfdd08d15f069b', 'mode': 'active'}]}


<ipython-input-18-1fc05361ae3b>:100: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token,  carousel_template_message)
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 04:17:20] "POST /callback HTTP/1.1" 200 -


建立 LINE bot 框架伺服器，回應 ImageCarousel template

In [23]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *
import json
import os, time, requests


def get_ngrokurl():
    os.system("ps aux | grep ngrok.exe | awk '{print $2}' | xargs kill -9")
    os.system("nohup ngrok http 5000&")
    time.sleep(3)
    public_url = json.loads(requests.get('http://localhost:4040/api/tunnels').text)['tunnels'][0]['public_url']
    return public_url

ngrok_url = get_ngrokurl()
print(ngrok_url)
# 你的 LINE Channel access token'
access_token = 'hea6hI/LTc4FCcgIlR51IbcAjPWqy9QtsG5XX9Ce+4GwdfJH1EKl5/YOT3Hw4+aZOGWZSKcKPmEnUqzE4tOoBjPdvyVJJjIHwm9RN9/cWbwT7Je2GW7uNtaTSKysDrcW8F/BSEFJpywQo5Z9DXry6gdB04t89/1O/w1cDnyilFU='
# 你的 LINE Channel access token
channel_secret = '77ff5430fad4531107ca11f1c495e7dd'
# Channel Access Token
line_bot_api = LineBotApi(access_token)
# Channel Secret
line_handler = WebhookHandler(channel_secret)
# 設定 webhook url
line_bot_api.set_webhook_endpoint(ngrok_url+'/callback')

# 建立 ImageCarousel 樣板中的卡片
column1 = ImageCarouselColumn(
                image_url = 'https://i.imgur.com/MMblDTQ.png',  # 呈現圖片
                action = PostbackAction(
                    label = '水族箱打氣機',
                    text='開關切換',
                    data = 'switch1')
)
column2 = ImageCarouselColumn(
                image_url = 'https://i.imgur.com/MMblDTQ.png',  # 呈現圖片
                action = PostbackAction(
                    label = '水族箱電燈',
                    text='開關切換',
                    data =  'switch2')
)
# 建立 ImageCarousel 樣板
columns = [column1, column2] # 收集卡片
carousel_template = ImageCarouselTemplate(columns=columns)

# 監聽所有來自 LINE Bot Channel 的 Post Request
app = Flask(__name__)
# 監聽所有來自 LINE Bot Channel 的 Post Request
@app.route("/callback", methods=['POST'])
def linebot():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # 取得傳來的訊息文字
    body = request.get_data(as_text=True)
    # 轉換訊息內容成 JSON 物件
    json_data = json.loads(body)
    print(f"訊息內容 {json_data}")
    # 驗證訊息正確性
    try:
        line_handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 處理訊息
@line_handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if event.message.text == 'imagecarousel':
      # 建立 Carousel 樣板訊息
      carousel_template_message = TemplateSendMessage(
        alt_text='Carousel Template 訊息',
        template = carousel_template
      )
      line_bot_api.reply_message(event.reply_token,  carousel_template_message)

# 處理 Postback 事件
@line_handler.add(PostbackEvent)
def handle_postback_message(event):
    postback = event.postback.data # 取得 Postback 事件的資料
# -------------------------------------
    if postback == 'switch1':
       message = TextSendMessage(text='水族箱打氣機開關已切換!')
    if postback == 'switch2':
       message = TextSendMessage(text='水族箱電燈開關已切換')
    line_bot_api.reply_message(event.reply_token, message)

if __name__ == "__main__":
  app.run()

https://4a28-34-83-245-156.ngrok-free.app


<ipython-input-23-8d852a6ed593>:24: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
<ipython-input-23-8d852a6ed593>:26: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_handler = WebhookHandler(channel_secret)
<ipython-input-23-8d852a6ed593>:28: LineBotSdkDeprecatedIn30: Call to deprecated method set_webhook_endpoint. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).set_webhook_endpoint(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_a

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470380263998750803', 'text': 'imagecarousel'}, 'webhookEventId': '01H8X94VV7VRH4HR2TY449C9PJ', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693199789484, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': '1d609fbdb1294abdbb625f1c3f3c159c', 'mode': 'active'}]}


...

訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470380270843855189', 'text': '開關切換'}, 'webhookEventId': '01H8X94ZT1K2G973RGFV80H37J', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693199793543, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': '519a97a34ef548958e24776edbdc831b', 'mode': 'active'}]}
訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'postback', 'postback': {'data': 'switch2'}, 'webhookEventId': '01H8X953Y4YHP37AZH7494Q6FZ', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693199798013, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': 'b6721dda800341e084e3b8b4e5861c2c', 'mode': 'active'}, {'type': 'message', 'message': {'type': 'text', 'id': '470380278209314930', 'text': '開關切換'}, 'webhookEventId': '01H8X953Y5V8S34K89J1A7J472', 'deliveryContext': {'isRedelivery': False}, '

INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 05:16:39] "POST /callback HTTP/1.1" 200 -


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'postback', 'postback': {'data': 'switch2'}, 'webhookEventId': '01H8X9GC81F7MZ5W3B8D832C1X', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693200166899, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': '01ff9c998b3a4d0ca7a713ae9ee32108', 'mode': 'active'}]}


INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 05:22:48] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Aug/2023 05:22:48] "POST /callback HTTP/1.1" 200 -


訊息內容 {'destination': 'Uef68021487469a005636e72e466f9acc', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '470380897071006148', 'text': '開關切換'}, 'webhookEventId': '01H8X9GC83G7CD7PTG0VG3Z3AF', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1693200166900, 'source': {'type': 'user', 'userId': 'Ubf2b9f4188d45848fb4697d41c962591'}, 'replyToken': 'f3d5c71cb7314495b4248138604e576c', 'mode': 'active'}]}
